# Secret in AWS Parameter Store

Parameter store is a Free service allows you to securely store parameters. This is how you deploy parameter:

In [12]:
import boto3
from pysecret import Parameter, deploy_parameter, delete_parameter

ssm_client = boto3.client("ssm")

param_name = "pysecret-demo"

_ = delete_parameter(ssm_client, param_name) # ensure it is not exists at beginning

Create a new parameter

In [13]:
param = deploy_parameter(
    ssm_client,
    name=param_name,
    data={"password": "mypassword1"},
    type_is_secure_string=True,
    use_default_kms_key=True,
    tier_is_intelligent=True,
    tags=dict(EnvName="dev"),
)
param

Parameter(Name='pysecret-demo', Type='SecureString', Value='{"password": "mypassword1"}', Version=1, LastModifiedDate=datetime.datetime(2023, 2, 11, 21, 47, 52), DataType='text', ARN=None, Selector=None, SourceResult=None)

Update a parameter

In [14]:
param = deploy_parameter(
    ssm_client,
    name=param_name,
    data={"password": "mypassword2"},
    type_is_secure_string=True,
    use_default_kms_key=True,
    tier_is_intelligent=True,
    tags=dict(EnvName="prod"), # also update tag
    overwrite=True, # set overwrite = True to allow update
    skip_if_duplicated=True, # if True, then it won't update if the parameter data is the same
)
param

Parameter(Name='pysecret-demo', Type='SecureString', Value='{"password": "mypassword2"}', Version=2, LastModifiedDate=datetime.datetime(2023, 2, 11, 21, 47, 53), DataType='text', ARN=None, Selector=None, SourceResult=None)

This is how you read parameter data

In [15]:
param = Parameter.load(ssm_client, name=param_name, with_decryption=True)
param

Parameter(Name='pysecret-demo', Type='SecureString', Value='{"password": "mypassword2"}', Version=2, LastModifiedDate=datetime.datetime(2023, 2, 11, 16, 47, 52, 997000, tzinfo=tzlocal()), DataType='text', ARN='arn:aws:ssm:us-east-1:669508176277:parameter/pysecret-demo', Selector=None, SourceResult=None)

In [16]:
# access the underlying user data
param.json_dict

{'password': 'mypassword2'}

``pysecret`` can even store arbitrary python object in AWS Parameter Store, if you installed ``jsonpickle`` first.

In [17]:
import dataclasses

@dataclasses.dataclass
class Env:
    username: str = dataclasses.field()
    password: str = dataclasses.field()


@dataclasses.dataclass
class Config:
    dev: Env = dataclasses.field()
    prod: Env = dataclasses.field()

In [18]:
config = Config(
    dev=Env(
        username="dev-username",
        password="dev-password",
    ),
    prod=Env(
        username="prod-username",
        password="prod-password",
    ),
)

param_name = "pysecret-pyobject"

deploy_parameter(
    ssm_client,
    name=param_name,
    data=config,
    type_is_string=True,
    tier_is_intelligent=True,
    overwrite=True,
)

param = Parameter.load(ssm_client, name=param_name)
param.py_object # access the underlying data

Config(dev=Env(username='dev-username', password='dev-password'), prod=Env(username='prod-username', password='prod-password'))

You can delete the parameters to keep your AWS Account clean

In [20]:
delete_parameter(ssm_client, "pysecret-demo");
delete_parameter(ssm_client, "pysecret-pyobject");